In [105]:
import torch
from torch import nn, optim
import os

from data_preprocess import *
from model import *
from utils import *

In [106]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() and torch.backends.mps.is_built() else 'cpu')
# print(device)

In [107]:
data_dir = 'dakshina_dataset_v1.0'
lang = 'hi'  # Hindi
subfolder_dir = 'lexicons'
train_path = os.path.join(data_dir, lang, subfolder_dir, f'{lang}.translit.sampled.train.tsv')
dev_path = os.path.join(data_dir, lang, subfolder_dir, f'{lang}.translit.sampled.dev.tsv')
test_path = os.path.join(data_dir, lang, subfolder_dir, f'{lang}.translit.sampled.test.tsv')

train_loader, val_loader, test_loader, src_vocab, tgt_vocab = prepare_dataloaders(train_path, dev_path, test_path, batch_size=2, repeat_datapoints=True)

# batch = next(iter(train_loader))
counter = 0
for src_batch, tgt_batch in train_loader:
    src = src_batch[0]
    tgt = tgt_batch[0]
    print(tgt_batch[0])
    print(tgt_batch.shape)
    print(src_vocab.decode(src))
    print(tgt_vocab.decode(tgt))
    counter += 1
    if counter>0:
        break

tensor([ 1, 32, 19, 24, 32, 34, 21, 13,  9,  6,  2])
torch.Size([2, 11])
vyavaharik
व्यवहारिक


In [123]:
len(tgt_vocab)

67

In [108]:
embedding_dim = 64
hidden_dim = 256
num_layers = 1
rnn_type = 'LSTM'  # can be 'RNN' or 'LSTM' or 'GRU'
num_epochs = 2
learning_rate = 0.0001

# Initialize models
encoder = Encoder(input_dim=len(src_vocab), emb_dim=embedding_dim, hidden_dim=hidden_dim,
                  num_layers=num_layers, rnn_type=rnn_type)#.to(device)
decoder = Decoder(output_dim=len(tgt_vocab), emb_dim=embedding_dim, hidden_dim=hidden_dim,
                  num_layers=num_layers, rnn_type=rnn_type)#.to(device)
model = Seq2Seq(encoder, decoder, device)#.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab.pad_idx)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [109]:
model.encoder.train()
model.decoder.train()
total_loss = 0

_, hidden = encoder(src_batch)
input_token = tgt_batch[:,0]
batch_size = src_batch.size(0)
tgt_len = tgt_batch.size(1)
output_dim = model.decoder.output_dim
outputs = torch.zeros(batch_size, tgt_len, output_dim)
print(outputs.shape)

torch.Size([2, 11, 67])


In [115]:
# Config
batch_size = 2
seq_len = 4
vocab_size = 10
embedding_dim = 8
hidden_dim = 16
num_layers = 2
cell_type = 'lstm'  # change to 'gru' or 'rnn' to test others

# Dummy input (each number is a token index)
src = torch.tensor([
    [1, 2, 3, 4],
    [5, 6, 7, 0]
])  # shape: (batch_size, seq_len)

print(f"Input shape: {src.shape} — [batch_size, seq_len]")

# Embedding layer
embedding = nn.Embedding(vocab_size, embedding_dim)
embedded = embedding(src)
print(f"Embedded shape: {embedded.shape} — [batch_size, seq_len, emb_dim]")

# RNN cell
if cell_type == 'lstm':
    rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
elif cell_type == 'gru':
    rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True)
else:
    rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)

# Forward pass
if cell_type == 'lstm':
    outputs, (hidden, cell) = rnn(embedded)
else:
    outputs, hidden = rnn(embedded)

print(f"RNN outputs shape: {outputs.shape} — [batch_size, seq_len, hidden_dim]")
print(f"Hidden shape: {hidden.shape} — [num_layers, batch_size, hidden_dim]")

if cell_type == 'lstm':
    print(f"Cell shape: {cell.shape} — [num_layers, batch_size, hidden_dim]")

Input shape: torch.Size([2, 4]) — [batch_size, seq_len]
Embedded shape: torch.Size([2, 4, 8]) — [batch_size, seq_len, emb_dim]
RNN outputs shape: torch.Size([2, 4, 16]) — [batch_size, seq_len, hidden_dim]
Hidden shape: torch.Size([2, 2, 16]) — [num_layers, batch_size, hidden_dim]
Cell shape: torch.Size([2, 2, 16]) — [num_layers, batch_size, hidden_dim]


In [116]:
import torch
import torch.nn as nn

# Config
batch_size = 2
vocab_size = 10
embedding_dim = 8
hidden_dim = 16
output_dim = vocab_size  # usually same as target vocab size
num_layers = 2
cell_type = 'lstm'  # change to 'gru' or 'rnn' to test others

# Dummy input token indices for a single time step
input_token = torch.tensor([1, 5])  # shape: (batch_size,)
print(f"Input token indices shape: {input_token.shape}")

# Embedding layer
embedding = nn.Embedding(vocab_size, embedding_dim)
embedded = embedding(input_token.unsqueeze(1))  # shape: (batch_size, 1, emb_dim)
print(f"Embedded shape: {embedded.shape} — [batch_size, 1, emb_dim]")

# RNN cell
if cell_type == 'lstm':
    rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
elif cell_type == 'gru':
    rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True)
else:
    rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)

# Dummy initial hidden state
if cell_type == 'lstm':
    hidden = (
        torch.zeros(num_layers, batch_size, hidden_dim),  # h_n
        torch.zeros(num_layers, batch_size, hidden_dim),  # c_n
    )
else:
    hidden = torch.zeros(num_layers, batch_size, hidden_dim)  # h_n

# Forward pass
if cell_type == 'lstm':
    output, (hidden, cell) = rnn(embedded, hidden)
else:
    output, hidden = rnn(embedded, hidden)

print(f"RNN output shape: {output.shape} — [batch_size, 1, hidden_dim]")
print(f"Hidden state shape: {hidden.shape} — [num_layers, batch_size, hidden_dim]")

# Final linear projection to vocabulary
fc_out = nn.Linear(hidden_dim, output_dim)
prediction = fc_out(output.squeeze(1))  # shape: (batch_size, output_dim)
print(f"Prediction shape: {prediction.shape} — [batch_size, output_dim]")

Input token indices shape: torch.Size([2])
Embedded shape: torch.Size([2, 1, 8]) — [batch_size, 1, emb_dim]
RNN output shape: torch.Size([2, 1, 16]) — [batch_size, 1, hidden_dim]
Hidden state shape: torch.Size([2, 2, 16]) — [num_layers, batch_size, hidden_dim]
Prediction shape: torch.Size([2, 67]) — [batch_size, output_dim]


In [124]:
for t in range(1, tgt_len):
    output, hidden = decoder(input_token, hidden)  # output: (batch_size, output_dim)
    outputs[:, t] = output
    
    teacher_force = torch.rand(1).item() < 0.2
    top1 = output.argmax(1)  # (batch_size)
    
    input_token = tgt_batch[:, t] if teacher_force else top1

RuntimeError: For batched 3-D input, hx and cx should also be 3-D but got (2-D, 2-D) tensors

In [112]:
outputs = outputs[:, 1:].reshape(-1, output_dim)
print(outputs.shape)
tgt_batch = tgt_batch[:, 1:].reshape(-1)
print(tgt_batch.shape)

torch.Size([20, 67])
torch.Size([20])


In [120]:
output.shape

torch.Size([2, 1, 16])